### `tes-thermo`
---

In [5]:
from tes_thermo.thermo_agent import ThermoAgent
from tes_thermo.utils import VectorSearch
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os

In [6]:
load_dotenv(override=True)

# example using Azure OpenAI services
# Make sure to set the environment variables AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT,
# LLM_DEPLOYMENT_MODEL, EMBEDDING_DEPLOYMENT_MODEL, LLM_API_VERSION, and EMBEDDING_API_VERSION
# in your .env file or your environment before running this code.

class AzureServices():
    def __init__(self):
        self.AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
        self.AZURE_OPENAI_ENDPOINT= os.getenv("AZURE_OPENAI_ENDPOINT")
        self.DEPLOYMENT_MODEL = "gpt-4.1-mini"
        self.EMBEGGIND_MODEL = os.getenv("EMBEDDING_DEPLOYMENT_MODEL")
        self.LLM_VERSION = os.getenv("LLM_API_VERSION")
        self.EMBEDDING_VERSION = os.getenv("EMBEDDING_API_VERSION")

    def get_default_llm(self):
        return AzureChatOpenAI(openai_api_version=self.LLM_VERSION,
                                azure_endpoint=self.AZURE_OPENAI_ENDPOINT,
                                openai_api_key=self.AZURE_OPENAI_API_KEY,
                                azure_deployment=self.DEPLOYMENT_MODEL,
                                temperature=0)
    
    def get_default_embedding(self):
        return AzureOpenAIEmbeddings(api_key=self.AZURE_OPENAI_API_KEY,
                                        api_version=self.EMBEDDING_VERSION,
                                        deployment=self.EMBEGGIND_MODEL,
                                        azure_endpoint=self.AZURE_OPENAI_ENDPOINT)

services = AzureServices()
llm = services.get_default_llm()
embedding_model = services.get_default_embedding()

In [9]:
%cd ..

/Users/mitoura/Desktop/Repositories/tes-thermo


In [10]:
vsearch = VectorSearch.from_documents(
    document_paths=['processes-13-02508.pdf'],  # replace with your document paths (pdf only...)
    embedding=embedding_model
)

Index created successfully with 65 chunks.


In [11]:
agent = ThermoAgent(llm = llm,
                    vsearch = None,
                    embedding_model= embedding_model)

In [12]:
prompt = """
Simulate the steam methane reforming (SMR) process. Consider methane, water, hydrogen, CO, and CO₂. 
The feed consists of 1 mol of water and 1 mol of CH₄ at 1200 K and 1 bar. 
Show the results as table.
Consulte as referencias e discute como otimizar esse processo.
"""
res = agent.chat(prompt=prompt)
print(res['output'])

Aqui está a simulação do processo de reforma a vapor do metano (SMR) para temperaturas entre 600 K e 1200 K, pressão de 1 bar, com alimentação de 1 mol de metano (CH4) e 1 mol de água (H2O). Os resultados mostram as frações molares dos componentes no equilíbrio:

| Temperatura (K) | CH4    | H2O    | H2      | CO      | CO2     |
|-----------------|--------|--------|---------|---------|---------|
| 600             | 0.9544 | 0.9090 | 0.1823  | 0.0003  | 0.0453  |
| 667             | 0.9089 | 0.8207 | 0.3614  | 0.0028  | 0.0882  |
| 733             | 0.8401 | 0.6971 | 0.6226  | 0.0168  | 0.1430  |
| 800             | 0.7409 | 0.5524 | 0.9657  | 0.0706  | 0.1885  |
| 867             | 0.6000 | 0.4085 | 1.3915  | 0.2085  | 0.1915  |
| 933             | 0.4239 | 0.2794 | 1.8728  | 0.4317  | 0.1444  |
| 1000            | 0.2589 | 0.1747 | 2.3075  | 0.6569  | 0.0842  |
| 1067            | 0.1453 | 0.1027 | 2.6068  | 0.8121  | 0.0426  |
| 1133            | 0.0807 | 0.0598 | 2.7788  | 0.8984  